In [1]:
from datasets import load_dataset, Dataset

/Users/ogu/source/revosax-embeddings/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
revosax = load_dataset("csv", data_files="../../data/training/training-data.csv", split="train")
revosax = revosax.rename_column("result", "query").rename_column("chunk", "answer")
revosax = revosax.select_columns(['query', 'answer'])
revosax = revosax.train_test_split(test_size=0.2, seed=12)

train_dataset: Dataset = revosax["train"]
eval_dataset: Dataset = revosax["test"]

Generating train split: 60789 examples [00:00, 90229.95 examples/s] 


In [3]:
texts = train_dataset["answer"][:5000]

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("../../training/models/german-nq-granite-embedding-278m-multilingual/checkpoint-4560")


/Users/ogu/source/revosax-embeddings/.venv/lib/python3.13/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [5]:
result = model.encode(texts, show_progress_bar=True, batch_size=32)


Batches: 100%|██████████| 157/157 [00:45<00:00,  3.41it/s]


In [6]:

# save embeddings to a file where each line is a list of floats, separated by tabs
with open('embeddings.tsv', 'w') as f:
    for row in result:
        f.write('\t'.join(map(str, row)) + '\n')

In [7]:
with open('lines.txt', 'w', encoding='utf-8') as out:
    for text in texts:
        first = text.split("\n", 1)[0]
        out.write(first + "\n")